<style  type="text/css"> 
</style>

<b><center>
<span style="font-size: 24pt; line-height: 1.2">
COMS W4111-002 (Fall 2021)<br>Introduction to Databases
</span>
</center></b>
</span><br>
<p>
<i><center>
<span style="font-size: 20pt; line-height: 1.2">
Homework 2: Programming Implement a Simple Database Engine<br>15 Points
</span>
</center></i>

## This assignment is due October 22, 11:59 pm EDT

__Note:__ Please replace the information below with your last name, first name and UNI.<br><br>



<i>
<span style="font-size: 20pt; line-height: 1.2"; >
Gupta Tushar, tg2749
</span>
</i>

### Submission
1. File > Print Preview > Save as PDF...
2. Upload .pdf and .ipynb to GradeScope

**This assignment is due October 22, 11:59 pm EDT**


### Collaboration
- You may use any information you get in TA or Prof. Ferguson's office hours, from lectures or from recitations.
- You may use information that you find on the web.
- You are NOT allowed to collaborate with other students outside of office hours.

## Part 1: Written & SQL

### Written 
Please keep your answers brief.

1. Codd's Fourth Rule states that: The data base description is represented at the logical level in the same way as ordinary data, so that authorized users can apply the same relational language to its interrogation as they apply to the regular data. In two sentences please explain this rule and why it is so important.
  
The rule says the below points: 
- These metadata should also be stored as tables, rows and columns. It should also have access privileges, thus showing all properties of a database.  
- We should be able to access these metadata by using same query language that we use to access the database.

It is important to ensure easy and standardised access/storage of database metadata without the need for additional tools and applications.  

2. Give 3 examples of what would be stored in a databsae catalog

Below are the 3 types of metadata information we can find in a database catalog:
- USER_PRIVILEGES: The different types of access and roles users have been given
- VIEWS : Any virtual queries that have been created
- COLUMN_PRIVILEGES: It details the types of access to different columns across tables in the database

3. What is the SQL database catalog called?

SQL database houses the metadata information and is called "information_schema" in MYSQL.

4. What is the overall goal of indicies in SQL?

A index is used to make retrieving data from the table faster. It improves the performance of queries and applications. It is basically a quick lookup table for finding records users need to search frequently. Databases automatically create indexes to process primary, foreign and unique key constraints faster.
    

5. What are the differences between a primary key and a unique index?

Unique key: This type of key is used to uniquely identify a tuple in the table and differs from the primary key as it allows a NULL value. However, all values are unique (thus only 1 NULL value). It is generally used when we want to impose unique constraints on a column or set of columns when they are not the primary key.
Primary key : By definition primary key only accepts unique values and cannot contain NULL. A table can contain only one primary key however it could be composed of multiple columns.
  

6. Which SELECT statement is more efficient? Why? 

- <code> SELECT playerID,birthState,nameLast,nameFirst FROM people 
where birthCountry = 'USA' and nameFirst = 'John' and playerID in (select playerID from collegeplaying where schoolID = 'Fordham'); </code>


- <code> SELECT playerID,birthState,nameLast,nameFirst FROM people NATURAL JOIN collegeplaying 
where birthCountry = 'USA' and nameFirst = 'John' and schoolID = 'Fordham' group by playerID,birthState,nameLast,nameFirst; </code>

HINT: SQL uses a query optimizer so you can't just run both of these and see which one performs faster.

The second query is faster as it uses a join on the 2 tables and basically goes over each of them exactly once. It then filters the resulting data to give the result. The time complexity is similar to O(m * n) 
However, the first query searches for schoolID in collegeplaying table for each row of people table. This can be lead to exponential complexity ( O(n^m) ).

7. The create.sql file provided in the zip folder makes a schema and some tables that mimics metadata tables. Note there is the sytax "ON DELETE CASCADE" after the foreign key creation. What does this mean? Why do we want to specify CASCADE for the metadata tables? What does "ON DELETE RESTRICT" mean and when would we generally want to use this? 

ON DELETE CASCADE contraint signfies that if a row in the Primary or the parent table is deleted it will automatically delete the corresponding value in the child table. It ensures there are no ambiguous scenarios, for eg: If we delete a column from the csvcolumns table, the corresponding row in the index should also be deleted as the column no longer exists. 

RESTRICT is the default behaviour of the foreign key where it will throw an error if we try to delete a primary table row which has a corresponding entry in the foreign key table. We have to manually first delete from the child table and then from the parent table. This is important to maintain the foreign key integrity constraints. Otherwise, it could lead to cases such that a column_name is deleted from the csvcolumns however, user forgot to delete the corresponding column_name row from the csvindexes table.

### SQL

#### 1. Initials

- Find the ```initials, firstName, lastName```, for every player from the people table.

- You need to return 10 rows.

- Sort by the nameFirst, nameLast ascending. 

- Note: Even for those players with two last names, just return the first letter of their first last name

<u>Answer:</u>

The below query adds a '$' str in front of the last name if the first name is None

In [36]:
import pymysql
import pandas as pd
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='admin123', db = 'lahmansbaseballdb')
pd.read_sql_query("select nameFirst as firstName, nameLast as lastName, CASE WHEN nameFirst IS NULL THEN CONCAT('$', LEFT(nameLast, 1)) ELSE CONCAT(LEFT(nameFirst, 1), LEFT(nameLast, 1)) END AS initials from lahmansbaseballdb.people order by nameFirst, nameLast limit 10;", conn)
        

,firstName,lastName,initials
0,None,Boland,$B
1,None,Booth,$B
2,None,Carroll,$C
3,None,Edwards,$E
4,None,Evans,$E
5,None,Franklin,$F
6,None,Gavern,$G
7,None,Harrison,$H
8,None,Hellings,$H
9,None,Higby,$H


## Question 1a): Games Per Player using GROUP BY

- Find the ```yearID, lgID, games_per_player```, for every year and league from the appearances table.

- Use a function to round down the games_per_player

- You need to return 10 rows.

- You must use `group by` in this query.

<u>Answer:</u>

In [ ]:
%%sql

In [37]:
pd.read_sql_query("select yearID, lgID, FLOOR(avg(G_all)) as games_per_player FROM appearances group by yearID, lgID limit 10;", conn)


,yearID,lgID,games_per_player
0,1871,NA,19
1,1872,NA,20
2,1873,NA,28
3,1874,NA,34
4,1875,NA,28
5,1876,NL,37
6,1877,NL,33
7,1878,NL,41
8,1879,NL,45
9,1880,NL,45


## Part 2: CSVCatalog Tests

Once you have tested everything successfuly in python, execute your tests one more time in jupyter notebook to show the expected output. You will need to restart your kernel after saving your python files so that jupyter will use the most recent version of your work. 

You may need to drop tables before executing your tests one last time so you don't run into integrity errors

In [1]:
import unit_test_catalog as cat # This notebook should be in the same directory as your project

In [2]:
cat.create_table_test()

Running save core definition
Q =  insert into csvtables values(%s, %s)
Running save core definition
Q =  insert into csvtables values(%s, %s)
in the index if statement
Table =  {"table_name": "test_table", "file_name": "./testing.csv", "indexes": []}
in the index if statement
Table =  {"table_name": "test_table2", "file_name": "./testing2.csv", "indexes": []}


In [3]:
cat.drop_table_test()

Q =  DELETE FROM csvtables WHERE table_name = 'test_table2'
Table 'test_table2' was dropped


In [4]:
cat.add_column_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)


In [5]:
cat.column_name_failure_test()  # This will throw an error

issue!!


ValueError: You must have a column name!!

In [6]:
cat.column_type_failure_test()  # This will throw an error

Issue!


ValueError: That column type is not accepted. Please try again.

In [7]:
cat.column_not_null_failure_test()  # This will throw an error

issue!


ValueError: The not_null column must be either True or False! Please try again.

In [8]:
cat.add_index_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvindexes (table_name, column_name, type, index_name, index_order)  values(%s, %s, %s, %s, %s)
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order


In [9]:
cat.col_drop_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  delete from csvcolumns where column_name = %s
Column 'player_age' has been dropped!
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order


In [10]:
cat.index_drop_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  DELETE FROM csvindexes WHERE table_name = 'test_table' and index_name = 'player_index'
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order


In [11]:
cat.describe_table_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
DESCRIBE People = 
 {
  "table_name": "test_table",
  "file_name": "./testing.csv",
  "columns": [
    {
      "column_name": "playerID",
      "column_type": "number",
      "not_null": true
    }
  ]
}


## Part 3: CSVTable Tests

In the event that the data sent is too large, jupyter notebook will throw a warning and not print any output. This will happen when you try to retrieve an entire table. Don't worry about getting the output if this happens. 

Additonally, the table formatting will get messed up if the columns makes the output too wide. In your tests make sure you project fields so that your outputs are legible. 

In [13]:
import unit_test_csv_table as tab

In [14]:
# Drop the tables if you already made them when testing
tab.drop_tables_for_prep()

Q =  DELETE FROM csvtables WHERE table_name = 'people'
Table 'people' was dropped
Q =  DELETE FROM csvtables WHERE table_name = 'batting'
Table 'batting' was dropped
Q =  DELETE FROM csvtables WHERE table_name = 'appearances'
Table 'appearances' was dropped


In [15]:
tab.create_lahman_tables()

Running save core definition
Q =  insert into csvtables values(%s, %s)
Running save core definition
Q =  insert into csvtables values(%s, %s)
Running save core definition
Q =  insert into csvtables values(%s, %s)


In [16]:
tab.update_people_columns()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert in

In [17]:
tab.update_appearances_columns()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert in

In [18]:
tab.update_batting_columns()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert into csvcolumns values(%s, %s, %s, %s)
Q =  insert in

In [19]:
tab.add_index_definitions()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvindexes (table_name, column_name, type, index_name, index_order)  values(%s, %s, %s, %s, %s)
Q =  insert into csvindexes (table_name, column_name, type, index_name, index_order)  values(%s, %s, %s, %s, %s)
Q =  insert into csvindexes (table_name, column_name, type, index_name, index_order)  values(%s, %s, %s, %s, %s)
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, ind

In [20]:
tab.test_load_info()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
Loaded  1000  rows
Loaded  1100  rows
Loaded  1200  rows
Loaded  1300  rows
Loaded  1400  rows
Loaded  1500  rows
Loaded  1600  rows
Loaded  1700  rows
Loaded  1800  rows
Loaded  1900  rows
Loaded  2000  rows
Loaded  2100  rows
Loaded  2200  rows
Loaded  2300  rows
Loaded  2400  rows
Loaded  2500  rows
Loaded  2600  rows
Loaded  2700  rows
Loaded  2800  rows
Loaded  2900  rows
Loaded  3000  rows
Loaded  3100  rows
Loaded  3200  rows
Loaded  3300  rows
Loaded  3400  rows
Loaded  3500  rows
Loaded  3600  rows
Loaded  3700  rows
Loaded  3800  rows
Loaded  3900  r

In [21]:
tab.test_get_col_names()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
Loaded  1000  rows
Loaded  1100  rows
Loaded  1200  rows
Loaded  1300  rows
Loaded  1400  rows
Loaded  1500  rows
Loaded  1600  rows
Loaded  1700  rows
Loaded  1800  rows
Loaded  1900  rows
Loaded  2000  rows
Loaded  2100  rows
Loaded  2200  rows
Loaded  2300  rows
Loaded  2400  rows
Loaded  2500  rows
Loaded  2600  rows
Loaded  2700  rows
Loaded  2800  rows
Loaded  2900  rows
Loaded  3000  rows
Loaded  3100  rows
Loaded  3200  rows
Loaded  3300  rows
Loaded  3400  rows
Loaded  3500  rows
Loaded  3600  rows
Loaded  3700  rows
Loaded  3800  rows
Loaded  3900  r

In [22]:
tab.add_other_indexes()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvindexes (table_name, column_name, type, index_name, index_order)  values(%s, %s, %s, %s, %s)
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Q =  insert into csvindexes (table_name, column_name, type, index_name, index_order)  values(%s, %s, %s, %s, %s)
Q =  insert into csvindexes (table_name, column_name, type, index_name, index_order)  values(%s, %s, %s,

In [23]:
# This should throw an error
# Make sure it works properly when you run it in pycharm though!
tab.load_test() 

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
+------+------+------+------+------+-----+--------+-----+-------+------+-------+--------+------------+-----+-------+------+------+------+------+---------+----------+----------+
|   2B |   3B |   AB |   BB |   CS |   G | GIDP   |   H | HBP   |   HR | IBB   | lgID   | playerID   |   R |   RBI |   SB | SF   | SH   |   SO |   stint | teamID   |   yearID |
+======+======+======+======+======+=====+========+=====+=======+======+=======+========+============+=====+=======+======+======+======+======+=========+==========+==========+
|    0 |    0 |    4 |    0 |    0 

## Please Note: I have reduced the sizes of batting and appearances as the code was taking too long to run. Both dumb_join() and smart_join() are tested on the same data to maintian consistency 

In [24]:
# Might throw an error depending on table size
# Make sure it works properly when you run it in pycharm though!
tab.dumb_join_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
{'2B': '0', '3B': '0', 'AB': '4', 'BB': '0', 'CS': '0', 'G': '1', 'GIDP': '', 'H': '0', 'HBP': '', 'HR': '0', 'IBB': '', 'lgID': 'NA

In [25]:
tab.get_access_path_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
{
  "index_name": "primary",
  "type": "PRIMARY",
  "columns": [
    "playerID",
    "yearID",
    "stint"
  ]
}
999


In [26]:
tab.sub_where_template_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
{'stint': '3', 'playerID': 'ras120', 'yearID': '34567'}


In [27]:
tab.test_find_by_template_index()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
[{'2B': '0', '3B': '0', 'AB': '4', 'BB': '0', 'CS': '0', 'G': '1', 'GIDP': '', 'H': '0', 'HBP': '', 'HR': '0', 'IBB': '', 'lgID': 'NA', 'playerID': 'abercda01', 'R': '0', 'RBI': '0', 'SB': '0', 'SF': '', 'SH': '', 'SO': '0', 'stint': '1', 'teamID': 'TRO', 'yearID': '1871'}]


In [28]:
tab.smart_join_test()

Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
{'2B': '0', '3B': '0', 'AB': '4', 'BB': '0', 'CS': '0', 'G': '1', 'GIDP': '', 'H': '0', 'HBP': '', 'HR': '0', 'IBB': '', 'lgID': 'NA

## Please Note: The below tests are run on reduced data size. We can observe that smart join performs 10X times faster on the same data. 

In [32]:
# Compare the time it takes to do the dumb join and the smart join below
 #This is a timer that will track how long it takes to execute your cell. 

# Times will vary based on how long it takes to query your AWS Server, but you should see a notable improvement using smart_join()

#----Your Code Here---
%time tab.dumb_join_test()


Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
{'2B': '0', '3B': '0', 'AB': '4', 'BB': '0', 'CS': '0', 'G': '1', 'GIDP': '', 'H': '0', 'HBP': '', 'HR': '0', 'IBB': '', 'lgID': 'NA

In [33]:
%time tab.smart_join_test()


Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
Loading core definition
Q =  select path from csvtables where table_name = %s
Loading columns
Q =  select * from csvcolumns where table_name = %s
Q =  select column_name, type, index_name, index_order from csvindexes where table_name = %s order by index_name, index_order
Loaded  100  rows
Loaded  200  rows
Loaded  300  rows
Loaded  400  rows
Loaded  500  rows
Loaded  600  rows
Loaded  700  rows
Loaded  800  rows
Loaded  900  rows
{'2B': '0', '3B': '0', 'AB': '4', 'BB': '0', 'CS': '0', 'G': '1', 'GIDP': '', 'H': '0', 'HBP': '', 'HR': '0', 'IBB': '', 'lgID': 'NA